In [1]:
from src.dataset import load_data, collect_samples
import argparse
import os
import requests
import tempfile
import numpy as np
import pandas as pd
import boto3

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from src.dataset import collect_samples, load_data

from src.utils import merge_two_dicts

In [3]:
columns_names = [
    "target_fg",
    "target_og",
    "ebc",
    "srm",
    "ph",
]
label_column = "ibu"

columns_dtype = {
    "target_fg": np.float64,
    "target_og": np.float64,
    "ebc": np.float64,
    "srm": np.float64,
    "ph": np.float64,
}
label_column_dtype = {"ibu": np.float64}

In [4]:
df = pd.read_csv('../data/dataset.csv')
df = df[columns_names + [label_column]]
df = df.dropna(subset=[label_column])

In [5]:
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]
)

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, columns_names),
    ]
)

In [6]:
y = df.pop(label_column)
X_pre = preprocess.fit_transform(df)
y_pre = y.to_numpy().reshape(len(y), 1)

X = np.concatenate((y_pre, X_pre), axis=1)

np.random.shuffle(X)
train, validation, test = np.split(X, [int(.7 * len(X)), int(.85 * len(X))])

In [7]:
pd.DataFrame(train).to_csv(f"../data/train.csv", header=False, index=False)
pd.DataFrame(validation).to_csv(f"../data/validation.csv", header=False, index=False)
pd.DataFrame(test).to_csv(f"../data/test.csv", header=False, index=False)

In [ ]:

framework_version = "0.23-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name="sklearn-abalone-process",
    role=role,
)